In [48]:
import warnings
warnings.filterwarnings("ignore")

In [131]:
from bs4 import BeautifulSoup
import urllib
query = input("Enter search query ")
query = query.replace(" ","+")
url = "http://www.google.com/search?q="+str(query)
print (url)
req = urllib.request.Request(url, headers={'User-Agent' : "Magic Browser"})
response = urllib.request.urlopen( req )
html = response.read()
soup = BeautifulSoup(html, 'html.parser')

Enter search query chandrayaan
http://www.google.com/search?q=chandrayaan


In [132]:
import re
all_links=[]
for link in soup.findAll('a'):
    all_links.append(link.get('href'))
    
imp_links = []
for x in all_links:
    if x.startswith("/url"):
        imp_links.append(x)
        
        
final_links = []
for x in imp_links:
    final_links.append("https://google.com"+x)

    
text=[]
for i in range(len(final_links)):
    req = urllib.request.Request(final_links[i],headers = {'User-Agent' : 'Magic Browser'})
    response = urllib.request.urlopen(req)
    html = response.read()
    soup = BeautifulSoup(html,'lxml')
    for j in range(len(soup.find_all('p'))):
        text.append(soup.find_all('p')[j].get_text())

        
#NLTK for cleaning
from nltk.corpus import stopwords
from string import punctuation
stuff_to_be_removed = list(stopwords.words("english"))+list(punctuation)
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

def my_tokenizer(s):
    s = s.lower()
    text = nltk.tokenize.word_tokenize(s)
    text = [t for t in text if len(t) > 2]
    text = [lemmatizer.lemmatize(y) for y in text if y not in stuff_to_be_removed]
    text = [t for t in text if not any(c.isdigit() for c in t)]
    text = " ".join(text)
    
    return text

final_text = [my_tokenizer(s) for s in text]
import pandas as pd
df = pd.DataFrame()
df["text"] = final_text
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
vector = tfidf.fit_transform(df["text"])
X = vector.toarray()

[nltk_data] Downloading package words to /Users/Arunabh/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [142]:
from sklearn.decomposition import TruncatedSVD
lsa = TruncatedSVD(n_components=10, n_iter=100)

In [143]:
lsa.fit(X)

TruncatedSVD(algorithm='randomized', n_components=10, n_iter=100,
             random_state=None, tol=0.0)

In [144]:
len(lsa.components_)

10

In [145]:
# ranking_topic_8 = lsa.components_[7].tolist()

In [146]:
# words = tfidf.get_feature_names()

In [147]:
# d = dict(zip(words,ranking_topic_8))

In [148]:
# d = pd.DataFrame(list(d.items()), columns=["words", "ranking"])

In [149]:
# d.sort_values(["ranking"], ascending=False)[:20]

In [150]:
words = tfidf.get_feature_names()
for i,y in enumerate(lsa.components_):
    componentwords = zip(words, y)
    sortedComponentwords = sorted(componentwords, key = lambda x: x[1],
                                reverse=True)
    sortedComponentwords = sortedComponentwords[:10]
    print ("\nTopic", i, ";")
    for x in sortedComponentwords:
        print (x)


Topic 0 ;
('lunar', 0.2754018597222927)
('orbit', 0.25367574278087124)
('moon', 0.25042511271278917)
('mission', 0.20376560270784969)
('spacecraft', 0.18566065058466558)
('isro', 0.17507702300115222)
('surface', 0.16750792895118627)
('space', 0.14976885203293208)
('launch', 0.13429571409077712)
('indian', 0.12630880001001388)

Topic 1 ;
('orbit', 0.36780065325325306)
('spacecraft', 0.24490913030790412)
('manoeuvre', 0.20612086178187422)
('minute', 0.19291099594590638)
('took', 0.19217340358212429)
('engine', 0.18488824761708864)
('raising', 0.179886004681796)
('fired', 0.17129620769867887)
('utc', 0.1643551435977019)
('half', 0.1597557022706967)

Topic 2 ;
('advertisement', 1.0)
('launch', 1.3015078961964956e-15)
('satellite', 1.2449710568688095e-15)
('orbit', 1.1831487590290802e-15)
('displayed', 8.672356732895626e-16)
('vehicle', 8.037853979210593e-16)
('gslv', 6.826786876673898e-16)
('spacecraft', 6.418171071172997e-16)
('india', 5.436858640311033e-16)
('space', 5.426052814688503e-